In [2]:
%pip install pyhwp

Note: you may need to restart the kernel to use updated packages.


In [3]:
import subprocess
from bs4 import BeautifulSoup
import re
import os
from typing import List, Dict, Any
import json
# projectmission2/notebooks/data_preprocessing_bjs(single,hwp5proc).ipynb

In [12]:
def convert_hwp_single_file(input_path, output_directory):
    """
    외부 hwp5proc 명령어를 사용하여 단일 HWP 파일을 XHTML로 변환합니다.

    Args:
        input_path (str): 변환할 HWP 파일의 전체 경로.
        output_directory (str): 변환된 파일을 저장할 디렉터리.
    """
    # 1. 출력 디렉터리 생성
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        print(f"출력 디렉터리가 없어 생성했습니다: {output_directory}")

    # 2. 출력 파일 경로 설정
    file_name = os.path.basename(input_path)
    base_name, _ = os.path.splitext(file_name)
    output_path = os.path.join(output_directory, f'{base_name}.xhtml')

    # 3. hwp5proc 명령어 및 인자 리스트 구성
    # 실제 hwp5proc 실행 파일 경로를 지정합니다. (사용자 환경에 맞게 수정 필요)
    hwp5proc_executable = "/home/spai0323/myenv/bin/hwp5proc" # <-- 이 경로를 실제 사용하시는 경로로 수정해주세요.
    command = [
        hwp5proc_executable,
        "xml",  # hwp5proc의 xml 하위 명령어 사용
        input_path
    ]

    print(f"HWP 파일 변환 시작: {input_path}")
    print(f"변환 결과 저장 경로: {output_path}")

    try:
        # 4. subprocess.run을 사용하여 명령어 실행
        result = subprocess.run(
            command,
            capture_output=True,
            text=True,
            check=True
        )

        # 5. 캡처된 출력을 파일에 쓰기
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(result.stdout)

        print("✅ 변환이 성공적으로 완료되었습니다.")
        print(f"   변환된 파일: {output_path}")

    except subprocess.CalledProcessError as e:
        print("❌ hwp5proc 실행 중 오류가 발생했습니다.")
        print(f"   명령어: {' '.join(e.cmd)}")
        print(f"   반환 코드: {e.returncode}")
        print(f"   표준 출력:\n{e.stdout}")
        print(f"   표준 오류:\n{e.stderr}")
    except FileNotFoundError:
        print(f"❌ 오류: hwp5proc 실행 파일을 찾을 수 없습니다. '{hwp5proc_executable}' 경로를 확인해주세요.")
    except Exception as e:
        print(f"❌ 예기치 않은 오류가 발생했습니다: {e}")

# --- 실행 부분 ---
# 변환할 단일 HWP 파일 경로
single_hwp_file = '../data/raw/files/대전대학교_대전대학교 2024학년도 다층적 융합 학습경험 플랫폼(MILE) 전.hwp'
# 변환된 XHTML 파일을 저장할 디렉터리
output_directory_single = '../data/processed/datapreprocessingbjs(hwp5proc)/single_xhtml' # 단일 파일 출력을 위한 별도 디렉터리

# 함수 호출
convert_hwp_single_file(single_hwp_file, output_directory_single)

출력 디렉터리가 없어 생성했습니다: ../data/processed/datapreprocessingbjs(hwp5proc)/single_xhtml
HWP 파일 변환 시작: ../data/raw/files/대전대학교_대전대학교 2024학년도 다층적 융합 학습경험 플랫폼(MILE) 전.hwp
변환 결과 저장 경로: ../data/processed/datapreprocessingbjs(hwp5proc)/single_xhtml/대전대학교_대전대학교 2024학년도 다층적 융합 학습경험 플랫폼(MILE) 전.xhtml
❌ hwp5proc 실행 중 오류가 발생했습니다.
   명령어: /home/spai0323/myenv/bin/hwp5proc xml ../data/raw/files/대전대학교_대전대학교 2024학년도 다층적 융합 학습경험 플랫폼(MILE) 전.hwp
   반환 코드: 1
   표준 출력:
<?xml version="1.0" encoding="utf-8"?>
<HwpDoc version="5.1.1.0"><HwpSummaryInfo><PropertySetStream byte-order="fffe" version="0" system-identifier="0000000d" clsid="9fa2b660-1061-11d4-b4c6-006097c09d8c"><PropertySet fmtid="9fa2b660-1061-11d4-b4c6-006097c09d8c" offset="48"><Property id="2" offset="120" id-label="PIDSI_TITLE" type="VT_LPWSTR" type-code="0x001f" value="붙임 10"></Property><Property id="3" offset="140" id-label="PIDSI_SUBJECT" type="VT_LPWSTR" type-code="0x001f" value=""></Property><Property id="4" offset="152" id-label="PIDSI_AUTHOR

In [5]:
def parse_hwp5proc_xhtml(file_path):
    """
    hwp5proc으로 변환된 XHTML 파일을 파싱하여 구조화된 데이터로 변환
    
    Args:
        file_path (str): XHTML 파일 경로
    
    Returns:
        dict: 파싱된 섹션과 표 데이터
    """
    print(f"XHTML 파일 파싱 시작: {file_path}")
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except UnicodeDecodeError:
        # UTF-8로 읽기 실패 시 다른 인코딩 시도
        with open(file_path, 'r', encoding='cp949') as f:
            content = f.read()
    
    soup = BeautifulSoup(content, 'xml')  # XML 파서 사용
    
    # 결과 저장용 딕셔너리
    parsed_data = {
        'sections': [],
        'tables': [],
        'metadata': {
            'total_paragraphs': 0,
            'total_tables': 0,
            'file_size': len(content)
        }
    }
    
    # 1. 표(TableControl) 추출
    tables = soup.find_all('TableControl')
    print(f"발견된 표 개수: {len(tables)}")
    
    for i, table in enumerate(tables):
        table_data = extract_table_structure(table, table_id=i)
        if table_data:
            parsed_data['tables'].append(table_data)
    
    # 2. 일반 텍스트 문단 추출
    paragraphs = soup.find_all('Paragraph')
    print(f"발견된 문단 개수: {len(paragraphs)}")
    
    for i, para in enumerate(paragraphs):
        text_content = extract_paragraph_text(para, para_id=i)
        if text_content and text_content.get('content', '').strip():
            parsed_data['sections'].append(text_content)
    
    # 메타데이터 업데이트
    parsed_data['metadata']['total_paragraphs'] = len(parsed_data['sections'])
    parsed_data['metadata']['total_tables'] = len(parsed_data['tables'])
    
    print(f"파싱 완료 - 문단: {len(parsed_data['sections'])}, 표: {len(parsed_data['tables'])}")
    return parsed_data

def extract_table_structure(table_element, table_id):
    """
    TableControl 요소에서 표 데이터 추출
    """
    try:
        table_data = {
            'id': table_id,
            'type': 'table',
            'rows': [],
            'raw_content': '',
            'metadata': {}
        }
        
        # TableBody 찾기
        table_body = table_element.find('TableBody')
        if not table_body:
            return None
            
        # TableRow들 찾기
        rows = table_body.find_all('TableRow')
        
        for row in rows:
            row_data = []
            cells = row.find_all('TableCell')
            
            for cell in cells:
                # 셀 내의 Text 요소들 추출
                cell_text = ""
                text_elements = cell.find_all('Text')
                for text_elem in text_elements:
                    if text_elem.string:
                        cell_text += text_elem.string.strip() + " "
                
                row_data.append(cell_text.strip())
            
            if any(row_data):  # 빈 행이 아닌 경우만 추가
                table_data['rows'].append(row_data)
        
        # 표 내용을 자연어로 변환
        table_data['raw_content'] = convert_table_to_natural_language(table_data['rows'])
        
        return table_data
        
    except Exception as e:
        print(f"표 추출 중 오류 발생: {e}")
        return None

def extract_paragraph_text(paragraph_element, para_id):
    """
    Paragraph 요소에서 텍스트 추출
    """
    try:
        text_content = ""
        
        # LineSeg 요소들 찾기
        line_segs = paragraph_element.find_all('LineSeg')
        
        for line_seg in line_segs:
            # Text 요소들 추출
            text_elements = line_seg.find_all('Text')
            for text_elem in text_elements:
                if text_elem.string:
                    text_content += text_elem.string.strip() + " "
        
        return {
            'id': para_id,
            'type': 'paragraph',
            'content': text_content.strip(),
            'metadata': {}
        }
        
    except Exception as e:
        print(f"문단 추출 중 오류 발생: {e}")
        return None

def convert_table_to_natural_language(table_rows):
    """
    표 데이터를 자연어 형태로 변환
    """
    if not table_rows or len(table_rows) < 2:
        return ""
    
    # 첫 번째 행을 헤더로 가정
    headers = table_rows[0]
    content_rows = table_rows[1:]
    
    natural_text = []
    
    # 추진일정표 특별 처리
    if any('월' in str(header) for header in headers) and '구분' in str(headers):
        natural_text.append("프로젝트 추진일정은 다음과 같습니다:")
        
        for row in content_rows:
            if len(row) > 1 and row[0].strip():  # 첫 번째 컬럼이 비어있지 않은 경우
                task = row[0].strip()
                # 각 월별 일정 정보 추출
                schedule_info = []
                for i, cell in enumerate(row[1:], 1):
                    if cell.strip() and i < len(headers):
                        month = headers[i]
                        schedule_info.append(f"{month}에 {task}")
                
                if schedule_info:
                    natural_text.extend(schedule_info)
    else:
        # 일반 표 처리
        for row in content_rows:
            if len(row) >= len(headers):
                row_text = []
                for i, (header, cell) in enumerate(zip(headers, row)):
                    if cell.strip():
                        row_text.append(f"{header}: {cell.strip()}")
                
                if row_text:
                    natural_text.append(", ".join(row_text))
    
    return ". ".join(natural_text)

# 실제 파일로 테스트 실행
def main():
    # 실제 경로
    xhtml_file_path = '../data/processed/datapreprocessingbjs(hwp5proc)/single_xhtml/(재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.xhtml'
    
    
    # 파일 존재 확인
    if not os.path.exists(xhtml_file_path):
        print(f"파일을 찾을 수 없습니다: {xhtml_file_path}")
        return
    
    # 파싱 실행
    parsed_result = parse_hwp5proc_xhtml(xhtml_file_path)
    
    # 결과 출력
    print("\n=== 파싱 결과 요약 ===")
    print(f"추출된 문단 수: {len(parsed_result['sections'])}")
    print(f"추출된 표 수: {len(parsed_result['tables'])}")
    print(f"파일 크기: {parsed_result['metadata']['file_size']:,} bytes")
    
    # 첫 번째 표 샘플 출력 (추진일정표)
    if parsed_result['tables']:
        print("\n=== 첫 번째 표 샘플 ===")
        first_table = parsed_result['tables'][0]
        print(f"표 ID: {first_table['id']}")
        print(f"행 수: {len(first_table['rows'])}")
        if first_table['raw_content']:
            print("자연어 변환 결과:")
            print(first_table['raw_content'][:500] + "..." if len(first_table['raw_content']) > 500 else first_table['raw_content'])
    
    # 첫 번째 문단 샘플 출력
    if parsed_result['sections']:
        print("\n=== 첫 번째 문단 샘플 ===")
        first_section = parsed_result['sections'][0]
        print(f"문단 ID: {first_section['id']}")
        content = first_section['content']
        print("내용:", content[:200] + "..." if len(content) > 200 else content)
    
    return parsed_result

# 실행
if __name__ == "__main__":
    result = main()

XHTML 파일 파싱 시작: ../data/processed/datapreprocessingbjs(hwp5proc)/single_xhtml/(재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.xhtml
발견된 표 개수: 95
발견된 문단 개수: 3161
파싱 완료 - 문단: 2007, 표: 95

=== 파싱 결과 요약 ===
추출된 문단 수: 2007
추출된 표 수: 95
파일 크기: 3,926,663 bytes

=== 첫 번째 표 샘플 ===
표 ID: 0
행 수: 2
자연어 변환 결과:
사 업 명: 주관기관, 통합 정보시스템 구축 사전 컨설팅: 재단법인 예술경영지원센터

=== 첫 번째 문단 샘플 ===
문단 ID: 5
내용: 제 안 요 청 서


In [6]:
def create_structured_chunks(parsed_data: Dict[str, Any], max_chunk_size: int = 1000) -> List[Dict[str, Any]]:
    """
    파싱된 데이터를 구조화된 청크로 변환
    
    Args:
        parsed_data: parse_hwp5proc_xhtml 함수의 결과
        max_chunk_size: 최대 청크 크기 (토큰 수 기준)
    
    Returns:
        List[Dict]: 구조화된 청크들의 리스트
    """
    chunks = []
    
    print("청킹 작업 시작...")
    
    # 1. 표 데이터 청킹
    for table in parsed_data['tables']:
        table_chunk = create_table_chunk(table)
        if table_chunk:
            chunks.append(table_chunk)
    
    # 2. 문단 데이터 청킹
    paragraph_chunks = create_paragraph_chunks(parsed_data['sections'], max_chunk_size)
    chunks.extend(paragraph_chunks)
    
    # 3. 청크에 고유 ID 부여
    for i, chunk in enumerate(chunks):
        chunk['chunk_id'] = f"chunk_{i:04d}"
    
    print(f"청킹 완료 - 총 {len(chunks)}개 청크 생성")
    return chunks

def create_table_chunk(table_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    표 데이터를 청크로 변환
    """
    if not table_data.get('raw_content') or not table_data['raw_content'].strip():
        return None
    
    # 표 유형 판단
    table_type = identify_table_type(table_data)
    
    chunk = {
        'type': 'table',
        'subtype': table_type,
        'content': table_data['raw_content'],
        'original_rows': len(table_data.get('rows', [])),
        'metadata': {
            'table_id': table_data.get('id'),
            'source': 'table_extraction',
            'priority': get_table_priority(table_type)
        }
    }
    
    return chunk

def identify_table_type(table_data: Dict[str, Any]) -> str:
    """
    표의 유형을 식별 (일정표, 예산표, 일반표 등)
    """
    content = table_data.get('raw_content', '').lower()
    rows = table_data.get('rows', [])
    
    # 헤더 행 확인
    if rows:
        header = ' '.join(rows[0]).lower()
        
        # 일정표 판단
        if any(keyword in header for keyword in ['월', '일정', '추진']):
            return 'schedule'
        
        # 예산표 판단
        if any(keyword in header for keyword in ['예산', '비용', '금액', '원']):
            return 'budget'
        
        # 조직표 판단
        if any(keyword in header for keyword in ['담당', '역할', '조직']):
            return 'organization'
    
    # 내용 기반 판단
    if any(keyword in content for keyword in ['일정', '월', '추진']):
        return 'schedule'
    elif any(keyword in content for keyword in ['예산', '비용', '천원']):
        return 'budget'
    else:
        return 'general'

def get_table_priority(table_type: str) -> int:
    """
    표 유형별 우선순위 설정
    """
    priority_map = {
        'schedule': 10,  # 일정표는 높은 우선순위
        'budget': 8,
        'organization': 6,
        'general': 4
    }
    return priority_map.get(table_type, 4)

def create_paragraph_chunks(sections: List[Dict[str, Any]], max_chunk_size: int) -> List[Dict[str, Any]]:
    """
    문단 데이터를 적절한 크기의 청크로 분할
    """
    chunks = []
    current_chunk = []
    current_size = 0
    
    for section in sections:
        content = section.get('content', '')
        if not content.strip():
            continue
        
        # 대략적인 토큰 수 계산 (한국어: 글자수 * 0.7)
        estimated_tokens = len(content) * 0.7
        
        # 현재 청크에 추가할 수 있는지 확인
        if current_size + estimated_tokens <= max_chunk_size:
            current_chunk.append(content)
            current_size += estimated_tokens
        else:
            # 현재 청크 저장하고 새 청크 시작
            if current_chunk:
                chunks.append(create_paragraph_chunk(current_chunk))
            
            # 새 청크 시작
            if estimated_tokens > max_chunk_size:
                # 너무 큰 문단은 분할
                split_chunks = split_large_paragraph(content, max_chunk_size)
                chunks.extend(split_chunks)
                current_chunk = []
                current_size = 0
            else:
                current_chunk = [content]
                current_size = estimated_tokens
    
    # 마지막 청크 처리
    if current_chunk:
        chunks.append(create_paragraph_chunk(current_chunk))
    
    return chunks

def create_paragraph_chunk(content_list: List[str]) -> Dict[str, Any]:
    """
    문단 리스트를 하나의 청크로 변환
    """
    combined_content = ' '.join(content_list)
    
    # 문단 유형 판단
    paragraph_type = identify_paragraph_type(combined_content)
    
    chunk = {
        'type': 'paragraph',
        'subtype': paragraph_type,
        'content': combined_content,
        'paragraph_count': len(content_list),
        'metadata': {
            'source': 'paragraph_extraction',
            'priority': get_paragraph_priority(paragraph_type)
        }
    }
    
    return chunk

def identify_paragraph_type(content: str) -> str:
    """
    문단의 유형을 식별
    """
    content_lower = content.lower()
    
    # 제목/헤더 판단
    if any(keyword in content_lower for keyword in ['제', '장', '절', '항']):
        return 'header'
    
    # 목적/개요 판단
    if any(keyword in content_lower for keyword in ['목적', '개요', '배경']):
        return 'overview'
    
    # 결론/요약 판단
    if any(keyword in content_lower for keyword in ['결론', '요약', '종합']):
        return 'conclusion'
    
    # 방법론 판단
    if any(keyword in content_lower for keyword in ['방법', '절차', '과정']):
        return 'methodology'
    
    return 'general'

def get_paragraph_priority(paragraph_type: str) -> int:
    """
    문단 유형별 우선순위 설정
    """
    priority_map = {
        'overview': 9,
        'conclusion': 8,
        'methodology': 7,
        'header': 6,
        'general': 5
    }
    return priority_map.get(paragraph_type, 5)

def split_large_paragraph(content: str, max_chunk_size: int) -> List[Dict[str, Any]]:
    """
    큰 문단을 여러 청크로 분할
    """
    chunks = []
    sentences = re.split(r'[.!?]\s+', content)
    
    current_chunk = []
    current_size = 0
    
    for sentence in sentences:
        estimated_tokens = len(sentence) * 0.7
        
        if current_size + estimated_tokens <= max_chunk_size:
            current_chunk.append(sentence)
            current_size += estimated_tokens
        else:
            if current_chunk:
                chunk_content = '. '.join(current_chunk) + '.'
                chunks.append({
                    'type': 'paragraph',
                    'subtype': 'split_general',
                    'content': chunk_content,
                    'paragraph_count': 1,
                    'metadata': {
                        'source': 'paragraph_split',
                        'priority': 5
                    }
                })
            
            current_chunk = [sentence]
            current_size = estimated_tokens
    
    # 마지막 청크
    if current_chunk:
        chunk_content = '. '.join(current_chunk) + '.'
        chunks.append({
            'type': 'paragraph',
            'subtype': 'split_general',
            'content': chunk_content,
            'paragraph_count': 1,
            'metadata': {
                'source': 'paragraph_split',
                'priority': 5
            }
        })
    
    return chunks

def analyze_chunks(chunks: List[Dict[str, Any]]) -> None:
    """
    생성된 청크들을 분석하고 통계를 출력
    """
    print("\n=== 청크 분석 결과 ===")
    
    # 타입별 통계
    type_counts = {}
    subtype_counts = {}
    
    for chunk in chunks:
        chunk_type = chunk['type']
        chunk_subtype = chunk['subtype']
        
        type_counts[chunk_type] = type_counts.get(chunk_type, 0) + 1
        subtype_counts[chunk_subtype] = subtype_counts.get(chunk_subtype, 0) + 1
    
    print("타입별 분포:")
    for chunk_type, count in type_counts.items():
        print(f"  {chunk_type}: {count}")
    
    print("\n세부 타입별 분포:")
    for subtype, count in subtype_counts.items():
        print(f"  {subtype}: {count}")
    
    # 우선순위별 분포
    priority_counts = {}
    for chunk in chunks:
        priority = chunk['metadata']['priority']
        priority_counts[priority] = priority_counts.get(priority, 0) + 1
    
    print("\n우선순위별 분포:")
    for priority in sorted(priority_counts.keys(), reverse=True):
        print(f"  우선순위 {priority}: {priority_counts[priority]}개")

# 실행 함수
def main_chunking(parsed_result):
    """
    파싱 결과를 받아서 청킹 실행
    """
    chunks = create_structured_chunks(parsed_result, max_chunk_size=1000)
    
    # 분석 결과 출력
    analyze_chunks(chunks)
    
    # 샘플 청크 출력
    print("\n=== 샘플 청크 ===")
    
    # 일정표 청크 찾기
    schedule_chunks = [c for c in chunks if c.get('subtype') == 'schedule']
    if schedule_chunks:
        print("일정표 청크:")
        sample_schedule = schedule_chunks[0]
        print(f"  타입: {sample_schedule['type']}-{sample_schedule['subtype']}")
        print(f"  내용: {sample_schedule['content'][:200]}...")
        print(f"  우선순위: {sample_schedule['metadata']['priority']}")
    
    # 일반 문단 청크
    paragraph_chunks = [c for c in chunks if c['type'] == 'paragraph']
    if paragraph_chunks:
        print("\n문단 청크:")
        sample_paragraph = paragraph_chunks[0]
        print(f"  타입: {sample_paragraph['type']}-{sample_paragraph['subtype']}")
        print(f"  내용: {sample_paragraph['content'][:200]}...")
        print(f"  문단 수: {sample_paragraph['paragraph_count']}")
    
    return chunks

chunks = main_chunking(result)

청킹 작업 시작...
청킹 완료 - 총 145개 청크 생성

=== 청크 분석 결과 ===
타입별 분포:
  table: 65
  paragraph: 80

세부 타입별 분포:
  general: 49
  budget: 2
  schedule: 15
  header: 62
  split_general: 17

우선순위별 분포:
  우선순위 10: 15개
  우선순위 8: 2개
  우선순위 6: 62개
  우선순위 5: 18개
  우선순위 4: 48개

=== 샘플 청크 ===
일정표 청크:
  타입: table-schedule
  내용: 구  분: 사업 주최 / 주관 / 발주, 주요 업무 내용: 사업총괄 관리 및 시행 사업계획서, 제안요청서 작성 및 주관사업자 선정 과제수행 관리 및 검사, 결과 평가. 구  분: 협조부서, 주요 업무 내용: 업체 선정 및 계약 진행 계약 관련 업무 진행 및 협조. 구  분: 사업자문, 주요 업무 내용: 미술시장 자문. 구  분: 연구팀, 주요 업무 내용: ...
  우선순위: 10

문단 청크:
  타입: paragraph-header
  내용: 제 안 요 청 서 사 업 명 통합 정보시스템 구축 사전 컨설팅 주관기관 재단법인 예술경영지원센터 사 업 명 통합 정보시스템 구축 사전 컨설팅 주관기관 재단법인 예술경영지원센터 사 업 명 통합 정보시스템 구축 사전 컨설팅 주관기관 재단법인 예술경영지원센터 2024년 04월 구 분 소 속 전화번호 이메일 입찰관련 경영지원팀 02-708-2212 ksj37@go...
  문단 수: 64


In [7]:
# 모든 파일 처리 결과 json 저장

output_dir = '../data/processed/datapreprocessingbjs(hwp5proc)'
output_file_path = os.path.join(output_dir, 'single_processed_data.json')

# 저장할 디렉터리가 없으면 생성합니다.
os.makedirs(output_dir, exist_ok=True)

try:
    with open(output_file_path, 'w', encoding='utf-8') as f:
        # 한글 깨짐을 방지하기 위해 ensure_ascii=False 옵션을 사용합니다.
        # indent=4는 가독성을 높여줍니다.
        json.dump(chunks, f, ensure_ascii=False, indent=4)
    print(f"'{output_file_path}'에 처리된 데이터가 성공적으로 저장되었습니다. ✅")
except Exception as e:
    print(f"파일 저장 중 오류가 발생했습니다: {e}")

'../data/processed/datapreprocessingbjs(간략화)/single_processed_data.json'에 처리된 데이터가 성공적으로 저장되었습니다. ✅


In [9]:
import json
import os
import chromadb
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# 기존 load_chunks_from_json 함수
def load_chunks_from_json(file_path):
    """JSON 파일에서 청크 데이터를 불러옵니다."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"✅ {file_path}에서 데이터를 성공적으로 불러왔습니다.")
        return data
    except FileNotFoundError:
        print(f"❌ 오류: 파일을 찾을 수 없습니다: {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"❌ 오류: JSON 디코딩에 실패했습니다. 파일 형식을 확인해주세요.")
        return None

# JSON 파일 경로 설정 및 데이터 로드
json_file_path = '../data/processed/datapreprocessingbjs(hwp5proc)/single_processed_data.json'
single_processed_data = load_chunks_from_json(json_file_path)

if single_processed_data is None:
    # 데이터 로드 실패 시, 스크립트 종료
    exit()

# # 모든 청크를 담을 단일 리스트 생성
# chunks = []
# for file_name, file_chunks in single_processed_data.items():
#     for chunk in file_chunks:
#         # 청크에 원본 파일 이름을 메타데이터로 추가
#         chunk['metadata']['source_file'] = file_name
#         chunks.append(chunk)

chunks = single_processed_data

print(f"총 {len(chunks)}개의 청크가 최종적으로 준비되었습니다.")

✅ ../data/processed/datapreprocessingbjs(간략화)/single_processed_data.json에서 데이터를 성공적으로 불러왔습니다.
총 145개의 청크가 최종적으로 준비되었습니다.


In [10]:
# chunks 리스트를 정상적으로 순회하며 청크 ID 고유화
for chunk in tqdm(chunks, desc="청크 ID 고유화 및 임베딩 준비"):
    file_name = chunk['metadata'].get("source_file", "unknown_file")
    # 기존 chunk_id가 없으면 새로 생성
    original_id = chunk.get('chunk_id', f"chunk_{len(chunks):04d}")
    # 파일 이름과 원래 ID를 결합하여 고유 ID 생성
    chunk['chunk_id'] = f"{os.path.basename(file_name)}_{original_id}"

청크 ID 고유화 및 임베딩 준비: 100%|██████████| 145/145 [00:00<00:00, 289331.15it/s]
